In [24]:
import pandas as pd
import numpy as np
import pickle
import yfinance as yf
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [25]:
# Mengambil data harga minyak mentah WTI dari Yahoo Finance
wti_data = yf.download('CL=F', start='2000-01-01', end='2023-12-29')
wti_data = wti_data[['Close']]
wti_data = wti_data.dropna()

[*********************100%%**********************]  1 of 1 completed


In [26]:
wti_data.tail()

Close
Date                 
2023-12-21  73.889999
2023-12-22  73.559998
2023-12-26  75.570000
2023-12-27  74.110001
2023-12-28  71.769997

In [27]:
#Memuat model
with open('model/crude-oil-lstm-fix2.pkl', 'rb') as f:
    lstm_model = pickle.load(f)
#Memuat model
with open('model/crude-oil-gru-fix2.pkl', 'rb') as f:
    gru_model = pickle.load(f)
#Memuat model
with open('model/crude-oil-prophet-fix.pkl', 'rb') as f:
    prophet_model = pickle.load(f)

In [31]:
# Menyimpan data untuk model LSTM dan GRU

#Mengubah data menjadi arrya numpy dan menskalanya
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(wti_data)

#fungsi untuk membuat dataset dengan sliding window
def create_dataset(data, time_step):
    X, Y = [],[]
    for i in range(len(data)-time_step-1):
        a = data[i:(i+time_step),0]
        X.append(a)
        Y.append(data[i + time_step,0])
    return np.array(X), np.array(Y)

#menyiapkan data untuk LSTM dan GRU
time_step = 3 #misalnya menggunakan 60 hari terakhir untuk memprediksi
X, y = create_dataset(scaled_data, time_step)

#mengubha bentuk data menjadi [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1],1)

In [32]:
scaled_data

array([[0.38093157],
       [0.37863548],
       [0.38093157],
       ...,
       [0.61884979],
       [0.61086817],
       [0.59807567]])

In [33]:
# Melakukan prediksi
def predict_future(model, data, n_days, time_step):
    predictions = []
    last_data = data[-time_step:]

    for i in range(n_days):
        last_data = last_data.reshape((1, time_step,1))
        pred = model.predict(last_data)
        predictions.append(pred[0,0])

        pred = np.array([pred[0, 0]]).reshape(1, 1, 1)
        last_data = np.append(last_data[:, 1:, :], pred, axis=1)

    return predictions

#prediksi untuk 7 dan 14 hari ke depan
lstm_predictions_7 = predict_future(lstm_model, scaled_data, 7, time_step)
lstm_predictions_14 = predict_future(lstm_model, scaled_data, 14, time_step)

gru_predictions_7 = predict_future(gru_model, scaled_data, 7, time_step)
gru_predictions_14 = predict_future(gru_model, scaled_data, 14, time_step)

#menginversi skala prediksi ke bentuk aslinya
lstm_predictions_7 = scaler.inverse_transform(np.array(lstm_predictions_7).reshape(-1,1))
lstm_predictions_14 = scaler.inverse_transform(np.array(lstm_predictions_14).reshape(-1,1))

gru_predictions_7 = scaler.inverse_transform(np.array(gru_predictions_7).reshape(-1,1))
gru_predictions_14 = scaler.inverse_transform(np.array(gru_predictions_14).reshape(-1,1))


1/1 [==============================] - 0s 72ms/step


In [34]:
lstm_predictions_7

array([[101.8142  ],
       [103.986374],
       [107.885506],
       [111.19804 ],
       [111.48116 ],
       [111.63099 ],
       [111.65888 ]], dtype=float32)

In [35]:
dates_7 = pd.date_range(start=wti_data.index[-1] + pd.Timedelta(days=1), periods=7, freq='D')
dates_14 = pd.date_range(start=wti_data.index[-1] + pd.Timedelta(days=1), periods=14, freq='D')

# Membuat DataFrame untuk prediksi
lstm_predictions_df_7 = pd.DataFrame(lstm_predictions_7, columns=['Predicted'], index=dates_7)
lstm_predictions_df_14 = pd.DataFrame(lstm_predictions_14, columns=['Predicted'], index=dates_14)

gru_predictions_df_7 = pd.DataFrame(gru_predictions_7, columns=['Predicted'], index=dates_7)
gru_predictions_df_14 = pd.DataFrame(gru_predictions_14, columns=['Predicted'], index=dates_14)

In [36]:
lstm_predictions_df_7

Predicted
2023-12-29  101.814201
2023-12-30  103.986374
2023-12-31  107.885506
2024-01-01  111.198036
2024-01-02  111.481163
2024-01-03  111.630989
2024-01-04  111.658882

In [37]:
lstm_predictions_df_14

Predicted
2023-12-29  101.814201
2023-12-30  103.986374
2023-12-31  107.885506
2024-01-01  111.198036
2024-01-02  111.481163
2024-01-03  111.630989
2024-01-04  111.658882
2024-01-05  111.660210
2024-01-06  111.660431
2024-01-07  111.660454
2024-01-08  111.660454
2024-01-09  111.660454
2024-01-10  111.660454
2024-01-11  111.660454

In [38]:
gru_predictions_df_7

Predicted
2023-12-29  41.879028
2023-12-30  37.346825
2023-12-31  31.710749
2024-01-01  24.957689
2024-01-02  22.434784
2024-01-03  20.121334
2024-01-04  18.274244

In [39]:
gru_predictions_df_14

Predicted
2023-12-29  41.879028
2023-12-30  37.346825
2023-12-31  31.710749
2024-01-01  24.957689
2024-01-02  22.434784
2024-01-03  20.121334
2024-01-04  18.274244
2024-01-05  17.264103
2024-01-06  16.468515
2024-01-07  15.902977
2024-01-08  15.543138
2024-01-09  15.279746
2024-01-10  15.097723
2024-01-11  14.975284

In [40]:
# Melakukan prediksi dengan prophet mode;
import prophet
from prophet import Prophet

wti_data.reset_index(inplace=True)
wti_data.rename(columns={'Date' : 'ds', 'Close' : 'y'}, inplace=True)

# Pastikan data asli memiliki frekuensi harian
wti_data.set_index('ds', inplace=True)
wti_data = wti_data.asfreq('D')

# Melakukan prediksi dengan Prophet
#prophet_model.fit(wti_data)

# Tentukan tanggal awal untuk prediksi
#last_date = wti_data.index[-1]
last_date = wti_data.index[-1]
future_7 = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=24, freq='M')
future_14 = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=48, freq='M')
future_7 = prophet_model.make_future_dataframe(periods=24, freq='M')#, include_history=False, start_date=last_date + pd.DateOffset(days=1))
future_14 = prophet_model.make_future_dataframe(periods=48, freq='M')#, include_history=False, start_date=last_date + pd.DateOffset(days=1))

forecast_7 = prophet_model.predict(future_7)
forecast_14 = prophet_model.predict(future_14)

# Mengambil nilai prediksi
prophet_predictions_7 = forecast_7[['ds','yhat']].tail()
prophet_predictions_14 = forecast_14[['ds','yhat']].tail()

C:\Users\Rizal\AppData\Local\Temp\ipykernel_5576\2602903561.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_7 = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=24, freq='M')
C:\Users\Rizal\AppData\Local\Temp\ipykernel_5576\2602903561.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_14 = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=48, freq='M')
c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [41]:
prophet_predictions_7

ds       yhat
5170 2021-08-31  75.479421
5171 2021-09-30  74.965952
5172 2021-10-31  72.582560
5173 2021-11-30  72.537460
5174 2021-12-31  73.370151

In [42]:
prophet_predictions_14

ds       yhat
5194 2023-08-31  86.607509
5195 2023-09-30  81.961969
5196 2023-10-31  84.299381
5197 2023-11-30  83.742498
5198 2023-12-31  82.934935

In [43]:
test = yf.download('CL=F', start='2000-01-01', end='2023-12-29')



[*********************100%%**********************]  1 of 1 completed


In [44]:
test.index

DatetimeIndex(['2000-08-23', '2000-08-24', '2000-08-25', '2000-08-28',
               '2000-08-29', '2000-08-30', '2000-08-31', '2000-09-01',
               '2000-09-05', '2000-09-06',
               ...
               '2023-12-14', '2023-12-15', '2023-12-18', '2023-12-19',
               '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-26',
               '2023-12-27', '2023-12-28'],
              dtype='datetime64[ns]', name='Date', length=5863, freq=None)

In [45]:
test[test.index == "2023-12-29"]

Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []

In [46]:
prophet_predictions_14

ds       yhat
5194 2023-08-31  86.607509
5195 2023-09-30  81.961969
5196 2023-10-31  84.299381
5197 2023-11-30  83.742498
5198 2023-12-31  82.934935

In [47]:
lstm_predictions_df_14

Predicted
2023-12-29  101.814201
2023-12-30  103.986374
2023-12-31  107.885506
2024-01-01  111.198036
2024-01-02  111.481163
2024-01-03  111.630989
2024-01-04  111.658882
2024-01-05  111.660210
2024-01-06  111.660431
2024-01-07  111.660454
2024-01-08  111.660454
2024-01-09  111.660454
2024-01-10  111.660454
2024-01-11  111.660454

In [48]:
gru_predictions_df_14

Predicted
2023-12-29  41.879028
2023-12-30  37.346825
2023-12-31  31.710749
2024-01-01  24.957689
2024-01-02  22.434784
2024-01-03  20.121334
2024-01-04  18.274244
2024-01-05  17.264103
2024-01-06  16.468515
2024-01-07  15.902977
2024-01-08  15.543138
2024-01-09  15.279746
2024-01-10  15.097723
2024-01-11  14.975284

In [49]:
test.head()

Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2000-08-23  31.950001  32.799999  31.950001  32.049999  32.049999   79385
2000-08-24  31.900000  32.240002  31.400000  31.629999  31.629999   72978
2000-08-25  31.700001  32.099998  31.320000  32.049999  32.049999   44601
2000-08-28  32.040001  32.919998  31.860001  32.869999  32.869999   46770
2000-08-29  32.820000  33.029999  32.560001  32.720001  32.720001   49131